In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import statsmodels.api as sm

In [ ]:
df_w=pd.read_csv('Confirmed_Cases_Dataset.csv').rename(columns={'Unnamed: 0':'Date'})
df_w.set_index('Date',inplace=True)
df_w.index=pd.DatetimeIndex(df.index)
df_w_log=np.log(1+df_w)

df_daily=pd.read_csv('Confirmed_Cases_Daily.csv').rename(columns={'Unnamed: 0':'Date'}).set_index('Date')
df_daily.index=pd.DatetimeIndex(df_daily.index)
for i in df_daily.columns:
  df_daily[i][np.logical_or(df_daily[i]==df_daily.iloc[-1,:][i],
                                  df_daily[i]==df_daily[df_daily[i]!=df_daily.iloc[-1,:][i]].iloc[-1][i])]=None


df_daily_log=np.log(1+df_daily)

<ipython-input-477-7dec82293188>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
def rolling_window_estimation(log_series,window_size,dailyfreq_dataset):
  rolling_dates=[]
  rolling_doubling_times=[]
  ci_lower=[]
  ci_upper=[]

  for i in log_series.index[window_size:-1]:
    end_date=i
    start_date=log_series.index[int(np.where(log_series.index==i)[0])-window_size]
    window_data=dailyfreq_dataset.loc[ start_date : end_date ]
    X, y = sm.add_constant(np.arange(1,window_data.shape[0]+1)), window_data
    model=sm.OLS(y,X).fit()
    incercept, slope = model.params
    se_slope=model.bse[1]

    doubling_time = np.log(2) / ((1+slope)**7-1) #Convert the estimated growth rate into weekly measure
    se_adjusted=model.bse[1]*np.sqrt(7) #Adjust Standard Deviation by number of days in a week

    rolling_dates.append(i)
    rolling_doubling_times.append(doubling_time)



  return rolling_dates,rolling_doubling_times







In [ ]:
WS=12
diz={}
for c in df_w.columns:
  diz[c]=rolling_window_estimation(log_series=df_w_log[c],window_size=WS,dailyfreq_dataset=df_daily_log[c])[1]

Doubling_Time=pd.DataFrame(diz).set_index(df_w_log.index[WS:-1])
px.line(Doubling_Time)

Date
2023-03-05    5548487
2023-03-06    5548487
2023-03-07    5548487
2023-03-08    5548487
2023-03-09    5548487
Name: Greece, dtype: int64